# LangChain 으로 Amazon Bedrock 시작하기
- [LangChain: ChatBedrock 예시](https://python.langchain.com/docs/integrations/chat/bedrock/)
- 

In [8]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("../../.env")



True

In [9]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key=os.environ.get('LANGFUSE_SECRET_KEY'),
    secret_key=os.environ.get('LANGFUSE_PUBLIC_KEY'),
    host="https://us.cloud.langfuse.com"
)

## LangCahin 모델 오브젝트 생성 및 실행

In [10]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=dict(temperature=0),
    # other params...
)

In [11]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Korean. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages, config={"callbacks": [langfuse_handler]})
ai_msg



AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={'usage': {'prompt_tokens': 29, 'completion_tokens': 19, 'total_tokens': 48}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 29, 'completion_tokens': 19, 'total_tokens': 48}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-0db05af8-ea7e-412a-9b0d-d316d473a0b1-0', usage_metadata={'input_tokens': 29, 'output_tokens': 19, 'total_tokens': 48})

In [12]:
print(ai_msg.content)

프로그래밍을 좋아합니다.


## Chaining

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "korean",
        "input": "I love programming.",
    },
    config={"callbacks": [langfuse_handler]}
)

AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={'usage': {'prompt_tokens': 23, 'completion_tokens': 19, 'total_tokens': 42}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 23, 'completion_tokens': 19, 'total_tokens': 42}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-954b435e-18b8-45d2-ac53-e8acfc713431-0', usage_metadata={'input_tokens': 23, 'output_tokens': 19, 'total_tokens': 42})

## Bedrock Converse API

In [14]:
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0,
    max_tokens=None,
    # other params...
)

llm.invoke(messages, config={"callbacks": [langfuse_handler]})

AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'a084fb25-aa64-42d9-b45b-8b7f59298274', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 09 Mar 2025 08:17:24 GMT', 'content-type': 'application/json', 'content-length': '216', 'connection': 'keep-alive', 'x-amzn-requestid': 'a084fb25-aa64-42d9-b45b-8b7f59298274'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [581]}}, id='run-c0b99661-488a-4776-9ba2-9ffb955f4250-0', usage_metadata={'input_tokens': 29, 'output_tokens': 19, 'total_tokens': 48})

In [15]:
for chunk in llm.stream(messages, config={"callbacks": [langfuse_handler]}):
    print(chunk)

content=[] additional_kwargs={} response_metadata={} id='run-d9691e98-08fe-4174-9dfa-b3bdde76c388'
content=[{'type': 'text', 'text': '프', 'index': 0}] additional_kwargs={} response_metadata={} id='run-d9691e98-08fe-4174-9dfa-b3bdde76c388'
content=[{'type': 'text', 'text': '로', 'index': 0}] additional_kwargs={} response_metadata={} id='run-d9691e98-08fe-4174-9dfa-b3bdde76c388'
content=[{'type': 'text', 'text': '그', 'index': 0}] additional_kwargs={} response_metadata={} id='run-d9691e98-08fe-4174-9dfa-b3bdde76c388'
content=[{'type': 'text', 'text': '래', 'index': 0}] additional_kwargs={} response_metadata={} id='run-d9691e98-08fe-4174-9dfa-b3bdde76c388'
content=[{'type': 'text', 'text': '밍', 'index': 0}] additional_kwargs={} response_metadata={} id='run-d9691e98-08fe-4174-9dfa-b3bdde76c388'
content=[{'type': 'text', 'text': '을', 'index': 0}] additional_kwargs={} response_metadata={} id='run-d9691e98-08fe-4174-9dfa-b3bdde76c388'
content=[{'type': 'text', 'text': ' ', 'index': 0}] additiona

In [16]:
from langchain_core.output_parsers import StrOutputParser

chain = llm | StrOutputParser()

for chunk in chain.stream(messages, config={"callbacks": [langfuse_handler]}):
    print(chunk, end="|")

|프|로|그|래|밍|을| |좋|아|합|니|다|.||||